## Vegetation indexes to support the burned area delineation 


In [1]:
workflow = dict([('id', 'vegetation-index'),
                ('label', 'Vegetation index'),
                ('doc', 'Vegetation index processor')])


In [14]:
input_reference = dict([('id', 'input_reference'), 
                        ('label', 'EO product for vegetation index'),
                        ('doc', 'EO product for vegetation index'),
                        ('value', '/workspace/application-chaining/68nl6_bz/s2-pre'), 
                        ('type', 'Directory'),
                        ('scatter', 'True')])

In [15]:
aoi = dict([('id', 'aoi'), 
              ('label', 'Area of interest'),
              ('doc', 'Area of interest in WKT'),
              ('value', 'POLYGON((136.508 -36.108,136.508 -35.654,137.178 -35.654,137.178 -36.108,136.508 -36.108))'), 
              ('type', 'string')])

### Vegetation indexes

NBR = (NIR - SWIR22) / (NIR + SWIR22)
NDVI = (NIR - Red) / (NIR + Red)
NDWI = (NIR - SWIR16) / (NIR + SWIR16)

In [16]:
bands = [{'name': 'NBR',
          'common_name': 'nbr'}, 
         {'name': 'NDVI',
          'common_name': 'ndvi'},
         {'name': 'NDWI',
          'common_name': 'ndwi'},
        ]

In [27]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import Catalog, Collection, EOItem, MediaType, EOAsset, CatalogType
from time import sleep
from shapely.wkt import loads
gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_nbr/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
cat = Catalog.from_file(os.path.join(input_reference['value'], 'catalog.json'))

In [20]:
collection = next(cat.get_children())

In [21]:
item = next(collection.get_items())

In [22]:
for index, band in enumerate(item.bands):
   
    if band.common_name in ['swir16']:
 
       asset_swir16 = item.assets[band.name].get_absolute_href()

    if band.common_name in ['swir22']:
 
       asset_swir22 = item.assets[band.name].get_absolute_href()
        
    if band.common_name in ['nir']:
 
        asset_nir = item.assets[band.name].get_absolute_href()
    
    if band.common_name in ['red']:
 
        asset_red = item.assets[band.name].get_absolute_href()

In [23]:
vrt = '{0}.vrt'.format(item.id)

In [24]:
ds = gdal.BuildVRT(vrt,
                   [asset_red, asset_nir, asset_swir16, asset_swir22],
                   srcNodata=0,
                   xRes=10, 
                   yRes=10,
                   separate=True)

ds.FlushCache()

ds = None

del(ds)


In [25]:
tif = '{0}.tif'.format(item.id)

In [28]:
min_lon, min_lat, max_lon, max_lat = loads(aoi['value']).bounds

In [29]:
gdal.Translate(tif,
               vrt,
               outputType=gdal.GDT_Int16,
               projWin=[min_lon, max_lat, max_lon, min_lat],
               projWinSRS='EPSG:4326')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f541dae2f30> >

In [30]:
os.remove(vrt)

In [31]:
ds = gdal.Open(tif)
width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

In [32]:
red = ds.GetRasterBand(1).ReadAsArray()
nir = ds.GetRasterBand(2).ReadAsArray()
swir16 = ds.GetRasterBand(3).ReadAsArray()
swir22 = ds.GetRasterBand(4).ReadAsArray()

ds = None

del(ds)

In [33]:
nbr = np.zeros((height, width), dtype=np.uint)

nbr = 10000 * ((nir - swir22) / (nir + swir22))

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
swir22 = None

del(swir22)

In [35]:
ndvi = np.zeros((height, width), dtype=np.uint)

ndvi = 10000 * ((nir - red) / (nir + red))

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
red = None

del(red)

In [37]:
ndwi = np.zeros((height, width), dtype=np.uint)

ndwi = 10000 * ((nir - swir16) / (nir + swir16))

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
nir = swir16 = None

del(nir)

del(swir16)

In [39]:
#temp_name = '_NRB_{}.tif'.format(item.id)
#output_name = 'NRB_{}.tif'.format(item.id)

In [40]:
def cog(input_tif, output_tif,no_data=None):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                    '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                    '-co COMPRESS=LZW '))
    
    if no_data != None:
        translate_options = gdal.TranslateOptions(gdal.ParseCommandLine('-co TILED=YES ' \
                                                                        '-co COPY_SRC_OVERVIEWS=YES ' \
                                                                        '-co COMPRESS=LZW '\
                                                                        '-a_nodata {}'.format(no_data)))
    ds = gdal.Open(input_tif, gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    
    ds = None

    del(ds)
    
    ds = gdal.Open(input_tif)
    gdal.Translate(output_tif,
                   ds, 
                   options=translate_options)
    ds = None

    del(ds)
    
    os.remove('{}.ovr'.format(input_tif))
    os.remove(input_tif)


In [41]:
catalog = Catalog(id='catalog', description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog>

In [42]:
item_name = 'INDEX_{}'.format(item.id)

In [43]:
result_item = EOItem(id=item_name,
                   geometry=item.geometry,
                   bbox=item.bbox,
                   datetime=item.datetime,
                   properties={},
                   bands=bands,
                    gsd=10, 
                    platform=item.platform, 
                    instrument=item.instrument)

In [44]:
for index, veg_index in enumerate(['NBR', 'NDVI', 'NDWI']):

    temp_name = '_{}_{}.tif'.format(veg_index, item.id)
    output_name = '{}_{}.tif'.format(veg_index, item.id)

    driver = gdal.GetDriverByName('GTiff')
    
    output = driver.Create(temp_name, 
                           width, 
                           height, 
                           1, 
                           gdal.GDT_Int16)

    output.SetGeoTransform(input_geotransform)
    output.SetProjection(input_georef)
    output.GetRasterBand(1).WriteArray(nbr),

    output.FlushCache()

    sleep(5)
    
    output = None

    del(output)

    os.makedirs(os.path.join('stac-results', item_name),
                exist_ok=True)
    
    cog(temp_name, os.path.join('stac-results', item_name, output_name))

    result_item.add_asset(key=veg_index.lower(),
                          asset=EOAsset(href='./{}'.format(output_name), 
                          media_type=MediaType.GEOTIFF, 
                          title=bands[index]['name'],
                          bands=bands[index]))

In [45]:
catalog.add_items([result_item])

In [46]:
os.remove(tif)

In [47]:
catalog.normalize_and_save(root_href='stac-results',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [48]:
catalog.describe()

* <Catalog id=catalog>
  * <EOItem id=INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808>
